# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [54]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import gmaps
import os
import json

# Import API key
from api_keys import geoapify_key
gmaps.configure(api_key=geoapify_key)

In [55]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,grytviken,-54.2811,-36.5092,34.83,90,56,35.23,GS,1683474397
1,1,salalah,17.0151,54.0924,84.29,74,0,8.05,OM,1683474397
2,2,port isabel,26.0734,-97.2086,87.03,75,0,17.27,US,1683474397
3,3,sinop,41.7711,34.8709,49.66,95,100,5.77,TR,1683474398
4,4,rosario,-32.9468,-60.6393,70.83,47,0,6.91,AR,1683474266


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [56]:
%%capture --no-display

humidity = city_data_df["Humidity"].astype(float)

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
     geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City"
)

# Display the map
map_plot_1



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [61]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.dropna()
weather_df = city_data_df

weather_df = weather_df.loc[(weather_df["Max Temp"] < 80)
                                              & (weather_df["Max Temp"] > 70)]

weather_df = weather_df.loc[weather_df["Wind Speed"] < 10]

weather_df = weather_df.loc[(weather_df["Humidity"] < 80) 
                           & (weather_df["Humidity"] > 45)]
# Drop any rows with null values
weather_df = weather_df.dropna()


# Display sample data
weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,rosario,-32.9468,-60.6393,70.83,47,0,6.91,AR,1683474266
7,7,inhambane,-23.8650,35.3833,72.66,77,0,5.93,MZ,1683474399
14,14,antonio enes,-16.2325,39.9086,74.30,72,3,7.11,MZ,1683474400
37,37,mugia,43.1041,-9.2179,71.17,67,70,4.85,ES,1683474405
87,87,jamestown,42.0970,-79.2353,71.19,54,40,5.99,US,1683474415


### Step 3: Create a new DataFrame called `hotel_df`.

In [59]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.count()

City          37
Country       37
Lat           37
Lng           37
Humidity      37
Hotel Name    37
dtype: int64

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [49]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
   
    
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
rosario - nearest hotel: Fontana Suites
inhambane - nearest hotel: Hotel de Inhambane
antonio enes - nearest hotel: Hotel Quirimbas
mugia - nearest hotel: Solpor do Coído
jamestown - nearest hotel: DoubleTree Jamestown
monteagudo - nearest hotel: Residencial San José
camacupa - nearest hotel: No hotel found
richards bay - nearest hotel: Protea Hotel
cabo san lucas - nearest hotel: Comfort Rooms
brigantine - nearest hotel: The Sea Gull
grand gaube - nearest hotel: Coco Villa
tocopilla - nearest hotel: hotel Isidora
paraiso - nearest hotel: Cabinas
castillos - nearest hotel: A mi gente
monte cristo - nearest hotel: No hotel found
pisco - nearest hotel: La Portada
kill devil hills - nearest hotel: Mariner Days Inn & Suites
mournies - nearest hotel: Oasis
luau - nearest hotel: No hotel found
mount airy - nearest hotel: Knights Inn Mount Airy
tadine - nearest hotel: Hôtel Nengoné
arroyito - nearest hotel: MH HOTEL
isangel - nearest hotel: Tanna Lodge
karratha - nearest

,City,Country,Lat,Lng,Humidity,Hotel Name
4,rosario,AR,-32.9468,-60.6393,47,Fontana Suites
7,inhambane,MZ,-23.8650,35.3833,77,Hotel de Inhambane
14,antonio enes,MZ,-16.2325,39.9086,72,Hotel Quirimbas
37,mugia,ES,43.1041,-9.2179,67,Solpor do Coído
87,jamestown,US,42.0970,-79.2353,54,DoubleTree Jamestown
95,monteagudo,BO,-19.8167,-63.9833,60,Residencial San José
120,camacupa,AO,-12.0167,17.4833,51,No hotel found
133,richards bay,ZA,-28.7830,32.0377,76,Protea Hotel
137,cabo san lucas,MX,22.8909,-109.9124,60,Comfort Rooms
162,brigantine,US,39.4101,-74.3646,66,The Sea Gull


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [53]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 600,
    frame_height = 500,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)